In [1]:
import pandas as pd
import numpy as np
import itertools
import lightgbm as lgbm
import matplotlib.pyplot as plt

from script import *
from feature_engineering import *
import db_operations as dbop
from constants import *
import data_process as dp

import time

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
pd.set_option('display.max_rows',200)
idx = pd.IndexSlice
agg_operations =['mean','median','max','min','std','size']

In [3]:
start = 20160101
cursor = dbop.connect_db("sqlite3").cursor()

df_d = dbop.create_df(cursor, STOCK_DAY[TABLE], start=start)
df_d = dp.proc_stock_d(dp.prepare_stock_d(df_d))
df_d.drop(columns=['open0','high0','low0','vol0'],inplace=True)
df_d['pct'] = df_d.sort_index().groupby('code')['close'].pct_change()*100
df_d['amt']/=1e5
df_d = df_d.astype('float32')
print(df_d.columns)
print(df_d.shape)
df_d.info(memory_usage='deep')
df_d.tail()

select * from stock_day where date>=20160101
Index(['open', 'high', 'low', 'close', 'vol', 'amt', 'adj_factor', 'avg',
       'close0', 'avg0', 'pct'],
      dtype='object')
(3524754, 11)
<class 'pandas.core.frame.DataFrame'>
MultiIndex: 3524754 entries, (000001.SZ, 2016-01-04 00:00:00) to (688399.SH, 2020-04-23 00:00:00)
Data columns (total 11 columns):
open          float32
high          float32
low           float32
close         float32
vol           float32
amt           float32
adj_factor    float32
avg           float32
close0        float32
avg0          float32
pct           float32
dtypes: float32(11)
memory usage: 321.8 MB


open        high         low       close  \
code      date                                                         
688399.SH 2020-04-17   95.949997   95.949997   88.879997   92.000000   
          2020-04-20   95.790001  109.970001   95.199997  108.730003   
          2020-04-21  108.000000  118.199997  102.000000  114.440002   
          2020-04-22  108.000000  118.500000  107.489998  109.089996   
          2020-04-23  117.029999  128.880005  117.029999  128.880005   

                               vol       amt  adj_factor         avg  \
code      date                                                         
688399.SH 2020-04-17  24047.009766  2.208113         1.0   91.824860   
          2020-04-20  34636.898438  3.590274         1.0  103.654602   
          2020-04-21  33081.828125  3.580782         1.0  108.240166   
          2020-04-22  21584.169922  2.432959         1.0  112.719612   
          2020-04-23  36237.828125  4.511718         1.0  124.502983   

                          close0        avg0        pct  
code      date                                           
688399.SH 2020-04-17   92.000000   91.824860  -3.055853  
          2020-04-20  108.730003  103.654602  18.184780  
          2020-04-21  114.440002  108.240166   5.251539  
          2020-04-22  109.089996  112.719612  -4.674941  
          2020-04-23  128.880005  124.502983  18.140995

In [4]:
df_d_basic = dbop.create_df(cursor, STOCK_DAILY_BASIC[TABLE], start=start)
df_d_basic = dp.prepare_stock_d_basic(df_d_basic)
#
df_d_basic["pb*pe_ttm"] = df_d_basic["pb"] * df_d_basic["pe_ttm"]
df_d_basic["pb*pe"] = df_d_basic["pb"] * df_d_basic["pe"]

df_d_basic = df_d_basic.astype('float32')
print(df_d_basic.columns)
print(df_d_basic.shape)
df_d_basic.info(memory_usage='deep')
df_d_basic.tail()

select * from stock_daily_basic where date>=20160101
Index(['close', 'turnover_rate', 'turnover_rate_f', 'volume_ratio', 'pe',
       'pe_ttm', 'pb', 'ps', 'ps_ttm', 'total_share', 'float_share',
       'free_share', 'total_mv', 'circ_mv', 'pb*pe_ttm', 'pb*pe'],
      dtype='object')
(3366811, 16)
<class 'pandas.core.frame.DataFrame'>
MultiIndex: 3366811 entries, (000001.SZ, 2016-01-04 00:00:00) to (688399.SH, 2020-04-23 00:00:00)
Data columns (total 16 columns):
close              float32
turnover_rate      float32
turnover_rate_f    float32
volume_ratio       float32
pe                 float32
pe_ttm             float32
pb                 float32
ps                 float32
ps_ttm             float32
total_share        float32
float_share        float32
free_share         float32
total_mv           float32
circ_mv            float32
pb*pe_ttm          float32
pb*pe              float32
dtypes: float32(16)
memory usage: 218.6 MB


close  turnover_rate  turnover_rate_f  \
code      date                                                     
688399.SH 2020-04-17   92.000000      18.028299        18.028299   
          2020-04-20  108.730003      25.967699        25.967699   
          2020-04-21  114.440002      24.801800        24.801800   
          2020-04-22  109.089996      16.181900        16.181900   
          2020-04-23  128.880005      27.167900        27.167900   

                      volume_ratio          pe     pe_ttm      pb         ps  \
code      date                                                                 
688399.SH 2020-04-17          1.74   84.497902  60.448002  5.2551  23.376801   
          2020-04-20          2.14   99.863602  71.440300  6.2107  27.627800   
          2020-04-21          1.54  105.108002  75.192001  6.5369  29.078699   
          2020-04-22          0.81  100.194298  71.676804  6.2313  27.719299   
          2020-04-23          1.25  118.370499  84.679703  7.3617  32.747898   

                         ps_ttm  total_share  float_share   free_share  \
code      date                                                           
688399.SH 2020-04-17  18.532900       5862.0  1333.847046  1333.847046   
          2020-04-20  21.903099       5862.0  1333.847046  1333.847046   
          2020-04-21  23.053400       5862.0  1333.847046  1333.847046   
          2020-04-22  21.975599       5862.0  1333.847046  1333.847046   
          2020-04-23  25.962200       5862.0  1333.847046  1333.847046   

                         total_mv        circ_mv   pb*pe_ttm       pb*pe  
code      date                                                            
688399.SH 2020-04-17  539304.0000  122713.921875  317.660278  444.044891  
          2020-04-20  637375.2500  145029.187500  443.694275  620.222900  
          2020-04-21  670847.2500  152645.453125  491.522583  687.080505  
          2020-04-22  639485.5625  145509.375000  446.639648  624.340698  
          2020-04-23  755494.5625  171906.203125  623.386597  871.408081

In [5]:
df = df_d
periods = [5,10,20,30,40,50,60,70,80,120,250]
for k in periods:
    df['{:d}ma'.format(k)]=df.reset_index('code').groupby('code')['close'].rolling(k).mean()
    df['{:d}ma_avg'.format(k)]=df.reset_index('code').groupby('code')['amt'].rolling(k).sum()\
            /df.reset_index('code').groupby('code')['vol'].rolling(k).sum()*1e6
print(df.columns)
    
for col2 in ["{}ma".format(k) for k in periods]:
    col1 = 'close'
    df['{}/{}'.format(col1,col2)] = df[col1]/df[col2]

for col2 in ["{}ma_avg".format(k) for k in periods]:
    col1 = 'close'
    df['{}/{}'.format(col1,col2)] = df[col1]/df[col2]

print(df.columns)

Index(['open', 'high', 'low', 'close', 'vol', 'amt', 'adj_factor', 'avg',
       'close0', 'avg0', 'pct', '5ma', '5ma_avg', '10ma', '10ma_avg', '20ma',
       '20ma_avg', '30ma', '30ma_avg', '40ma', '40ma_avg', '50ma', '50ma_avg',
       '60ma', '60ma_avg', '70ma', '70ma_avg', '80ma', '80ma_avg', '120ma',
       '120ma_avg', '250ma', '250ma_avg'],
      dtype='object')
Index(['open', 'high', 'low', 'close', 'vol', 'amt', 'adj_factor', 'avg',
       'close0', 'avg0', 'pct', '5ma', '5ma_avg', '10ma', '10ma_avg', '20ma',
       '20ma_avg', '30ma', '30ma_avg', '40ma', '40ma_avg', '50ma', '50ma_avg',
       '60ma', '60ma_avg', '70ma', '70ma_avg', '80ma', '80ma_avg', '120ma',
       '120ma_avg', '250ma', '250ma_avg', 'close/5ma', 'close/10ma',
       'close/20ma', 'close/30ma', 'close/40ma', 'close/50ma', 'close/60ma',
       'close/70ma', 'close/80ma', 'close/120ma', 'close/250ma',
       'close/5ma_avg', 'close/10ma_avg', 'close/20ma_avg', 'close/30ma_avg',
       'close/40ma_avg', 'close/

In [6]:
start_date = '-'.join([str(start)[:4],str(start)[4:6],str(start)[6:8]])
df_r_spl = pd.read_parquet(r"database\return_spl_5%_10%_20_8_15%").loc[idx[:,start_date:],:].sort_index()
df_r_spc = pd.read_parquet(r"database\return_spc_5%_10%_20_8_15%").loc[idx[:,start_date:],:].sort_index()

features = []
for col2 in ["{}ma".format(k) for k in periods]+["{}ma_avg".format(k) for k in periods]:
    col1 = 'close'
    features.append('{}/{}'.format(col1,col2))
print(features)
# result_50 = ml.assess_feature3(df[features],df_r.r,q_bin=50)
stats_1000_spl,result_1000_spl = ml.assess_feature3(df[features],df_r_spl.r,q_bin=1000)
stats_1000_spc,result_1000_spc = ml.assess_feature3(df[features],df_r_spc.r,q_bin=1000)

# print(result_50.sort_values(['median_std','mean_std'],ascending=False))
stats_1000_spl.sort_values(['median_std','mean_std'],ascending=False)
stats_1000_spc.sort_values(['median_std','mean_std'],ascending=False)

C:\Users\dell-pc\Anaconda3.6\lib\site-packages\pyarrow\pandas_compat.py:707: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels = getattr(columns, 'labels', None) or [
C:\Users\dell-pc\Anaconda3.6\lib\site-packages\pyarrow\pandas_compat.py:734: FutureWarning: the 'labels' keyword is deprecated, use 'codes' instead
  return pd.MultiIndex(levels=new_levels, labels=labels, names=columns.names)
C:\Users\dell-pc\Anaconda3.6\lib\site-packages\pyarrow\pandas_compat.py:751: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels


['close/5ma', 'close/10ma', 'close/20ma', 'close/30ma', 'close/40ma', 'close/50ma', 'close/60ma', 'close/70ma', 'close/80ma', 'close/120ma', 'close/250ma', 'close/5ma_avg', 'close/10ma_avg', 'close/20ma_avg', 'close/30ma_avg', 'close/40ma_avg', 'close/50ma_avg', 'close/60ma_avg', 'close/70ma_avg', 'close/80ma_avg', 'close/120ma_avg', 'close/250ma_avg']
close/10ma 
                               mean    median
bin                                         
(0.58091284, 0.80099142] -0.025827 -0.002703
(0.80099142, 0.82385952]  0.028609  0.022740
(0.82385952, 0.83601029]  0.025604  0.019868
(0.83601029, 0.84453193]  0.027698  0.020730
(0.84453193, 0.85111993]  0.025466  0.017731
...                            ...       ...
(1.18429773, 1.20102894] -0.015562 -0.025253
(1.20102894, 1.22663032] -0.016074 -0.028436
(1.22663032, 1.26885383] -0.023223 -0.033956
(1.26885383, 1.3799625]  -0.020435 -0.036211
(1.3799625, 1.57867704]  -0.021914 -0.030309

[1000 rows x 2 columns] 

close/10ma_avg 
    

close/5ma_avg 
                                    mean    median
bin                                              
(0.6025900000000001, 0.84685] -0.003571  0.002857
(0.84685, 0.86221]             0.014566  0.009067
(0.86221, 0.87129]             0.018926  0.012587
(0.87129, 0.8779]              0.016883  0.006623
(0.8779, 0.88306]              0.011584  0.004609
...                                 ...       ...
(1.10646, 1.1149]             -0.014334 -0.025581
(1.1149, 1.12615]             -0.019679 -0.028872
(1.12615, 1.14292]            -0.019697 -0.032109
(1.14292, 1.17127]            -0.023979 -0.038540
(1.17127, 1.46888]            -0.030320 -0.044444

[1000 rows x 2 columns] 

close/60ma 
                       mean    median
bin                                 
(0.3089, 0.6601]  0.007460  0.000233
(0.6601, 0.6929]  0.015157  0.007509
(0.6929, 0.7106]  0.015763  0.006235
(0.7106, 0.7226]  0.015930  0.004532
(0.7226, 0.7321]  0.014376  0.004751
...                    ...       ..

close/30ma 
                       mean    median
bin                                 
(0.3689, 0.722]   0.009591 -0.008008
(0.722, 0.7523]   0.028447  0.005949
(0.7523, 0.7677]  0.029970  0.007475
(0.7677, 0.7783]  0.030885  0.006889
(0.7783, 0.7862]  0.028183  0.007527
...                    ...       ...
(1.2995, 1.3215] -0.014160 -0.033887
(1.3215, 1.3518] -0.015720 -0.035470
(1.3518, 1.3963] -0.021381 -0.040816
(1.3963, 1.4835] -0.016088 -0.040042
(1.4835, 2.3624] -0.021017 -0.046154

[1000 rows x 2 columns] 

close/30ma_avg 
                                    mean    median
bin                                              
(0.47950000000000004, 0.7222]  0.023253  0.004545
(0.7222, 0.7424]               0.024079  0.002632
(0.7424, 0.7548]               0.020117  0.000000
(0.7548, 0.764]                0.020944  0.002543
(0.764, 0.7709]                0.021326  0.000000
...                                 ...       ...
(1.2263, 1.2409]              -0.016164 -0.033602
(1.2409, 1.2

,mean_std,median_std,mean_max,median_max,mean_min,median_min,mean_q96%,median_q96%,mean_q4%,median_q4%
close/20ma,0.004031,0.004426,0.025207,0.017083,-0.020330,-0.033783,0.006901,-0.001949,-0.003762,-0.013845
close/10ma,0.004210,0.004418,0.028609,0.022740,-0.025827,-0.036211,0.004896,-0.002968,-0.004834,-0.014797
close/10ma_avg,0.003865,0.004242,0.024054,0.020467,-0.027875,-0.042918,0.004611,-0.003935,-0.004943,-0.014975
close/20ma_avg,0.003717,0.004036,0.022330,0.018097,-0.021790,-0.033505,0.007185,-0.002452,-0.003622,-0.013886
close/30ma,0.003781,0.003874,0.021325,0.013989,-0.022702,-0.034812,0.007232,-0.003584,-0.003952,-0.013832
close/40ma,0.003887,0.003839,0.018121,0.010543,-0.020380,-0.033058,0.008148,-0.002478,-0.004251,-0.014216
close/50ma,0.003821,0.003786,0.017210,0.008869,-0.018627,-0.032500,0.007967,-0.002567,-0.004413,-0.014416
close/60ma,0.003839,0.003756,0.015930,0.007509,-0.016826,-0.029941,0.008132,-0.001976,-0.004980,-0.014651
close/70ma,0.003765,0.003676,0.017388,0.007031,-0.014060,-0.027015,0.008315,-0.002108,-0.004819,-0.014475
close/120ma_avg,0.003712,0.003553,0.024860,0.014706,-0.011050,-0.017564,0.007926,-0.002468,-0.004027,-0.014057


,mean_std,median_std,mean_max,median_max,mean_min,median_min,mean_q96%,median_q96%,mean_q4%,median_q4%
close/5ma,0.016874,0.009474,0.372833,0.195580,-0.017985,-0.043591,0.005163,-0.012842,-0.005903,-0.020022
close/60ma,0.005995,0.005738,0.026530,0.002427,-0.016957,-0.042403,0.012759,-0.005725,-0.008273,-0.026375
close/50ma,0.005906,0.005615,0.027325,0.004662,-0.019031,-0.044951,0.012318,-0.006189,-0.007613,-0.025432
close/70ma,0.005835,0.005589,0.026934,0.003711,-0.016209,-0.039490,0.012975,-0.005176,-0.007870,-0.026459
close/40ma,0.006061,0.005516,0.028494,0.005545,-0.019861,-0.044923,0.013210,-0.006486,-0.007120,-0.024669
close/120ma,0.005955,0.005351,0.033021,0.004637,-0.018680,-0.035870,0.013556,-0.005246,-0.007434,-0.025732
close/80ma,0.005524,0.005304,0.027922,0.002141,-0.015529,-0.038436,0.012869,-0.005607,-0.007281,-0.026132
close/10ma,0.008284,0.005255,0.195663,0.058154,-0.019335,-0.044526,0.008821,-0.009214,-0.006414,-0.021130
close/30ma,0.005750,0.005212,0.030885,0.007527,-0.021381,-0.046154,0.011350,-0.008004,-0.006884,-0.024729
close/60ma_avg,0.005611,0.005205,0.022575,0.002287,-0.015729,-0.035885,0.011654,-0.007091,-0.007674,-0.025245


由上可知，avg类变量在等频分箱下差异更小，区分度弱于非avg，但在末端(1%)下的收益率明显高于非avg。

In [7]:
# import importlib
# import ml_model as ml
# importlib.reload(ml)


In [8]:
stats_1000_spl.sort_values(['mean_max'],ascending=False)
stats_1000_spc.sort_values(['mean_max'],ascending=False)

,mean_std,median_std,mean_max,median_max,mean_min,median_min,mean_q96%,median_q96%,mean_q4%,median_q4%
close/10ma,0.004210,0.004418,0.028609,0.022740,-0.025827,-0.036211,0.004896,-0.002968,-0.004834,-0.014797
close/250ma_avg,0.003247,0.002768,0.026932,0.015441,-0.009369,-0.018163,0.007178,-0.004921,-0.003755,-0.013771
close/20ma,0.004031,0.004426,0.025207,0.017083,-0.020330,-0.033783,0.006901,-0.001949,-0.003762,-0.013845
close/120ma_avg,0.003712,0.003553,0.024860,0.014706,-0.011050,-0.017564,0.007926,-0.002468,-0.004027,-0.014057
close/5ma,0.003412,0.003285,0.024293,0.012642,-0.036518,-0.045920,0.003547,-0.006476,-0.004434,-0.014137
close/10ma_avg,0.003865,0.004242,0.024054,0.020467,-0.027875,-0.042918,0.004611,-0.003935,-0.004943,-0.014975
close/20ma_avg,0.003717,0.004036,0.022330,0.018097,-0.021790,-0.033505,0.007185,-0.002452,-0.003622,-0.013886
close/80ma_avg,0.003347,0.003106,0.022287,0.011259,-0.013287,-0.022296,0.007208,-0.003788,-0.004258,-0.013975
close/30ma,0.003781,0.003874,0.021325,0.013989,-0.022702,-0.034812,0.007232,-0.003584,-0.003952,-0.013832
close/120ma,0.003709,0.003499,0.020073,0.007178,-0.012952,-0.022430,0.008592,-0.002265,-0.004550,-0.014347


,mean_std,median_std,mean_max,median_max,mean_min,median_min,mean_q96%,median_q96%,mean_q4%,median_q4%
close/5ma,0.016874,0.009474,0.372833,0.195580,-0.017985,-0.043591,0.005163,-0.012842,-0.005903,-0.020022
close/5ma_avg,0.010957,0.002766,0.233092,0.004136,-0.007141,-0.028545,0.004814,-0.012918,-0.005203,-0.020277
close/10ma,0.008284,0.005255,0.195663,0.058154,-0.019335,-0.044526,0.008821,-0.009214,-0.006414,-0.021130
close/10ma_avg,0.005921,0.004085,0.099654,0.012714,-0.011466,-0.035539,0.008420,-0.011124,-0.006191,-0.021516
close/250ma_avg,0.005231,0.004262,0.047410,0.019947,-0.012631,-0.028743,0.010979,-0.009200,-0.006068,-0.022942
close/120ma_avg,0.005882,0.005056,0.038664,0.011437,-0.012811,-0.032447,0.012714,-0.005339,-0.005947,-0.024260
close/20ma,0.005997,0.005135,0.036349,0.012866,-0.020720,-0.041508,0.011588,-0.007350,-0.005773,-0.023440
close/80ma_avg,0.005112,0.004665,0.036032,0.011981,-0.014251,-0.035926,0.011030,-0.007699,-0.006497,-0.024381
close/120ma,0.005955,0.005351,0.033021,0.004637,-0.018680,-0.035870,0.013556,-0.005246,-0.007434,-0.025732
close/30ma,0.005750,0.005212,0.030885,0.007527,-0.021381,-0.046154,0.011350,-0.008004,-0.006884,-0.024729


In [9]:
result_1000_spc['close/5ma'].tail(10)
mask = df['close/5ma']>1.199
index = df.index[mask]
len(index)
df_r_spc.loc[index].reset_index('code')['r'].resample('Q').agg(agg_operations)
df_r_spl.loc[index].reset_index('code')['r'].resample('Q').agg(agg_operations)
df_r_spl.loc[index].sort_values('date',ascending=False).head(20)


In [10]:
data_min_spc = [[col,df['mean'][0],df['median'][0]] for col,df in result_1000_spc.items()]
columns = ['col','mean','median']
df_min_spc = pd.DataFrame(data_min_spc,columns=columns)
df_min_spc.sort_values('mean',ascending=False)

data_min_spl = [[col,df['mean'][0],df['median'][0]] for col,df in result_1000_spl.items()]
df_min_spl = pd.DataFrame(data_min_spl,columns=columns)
df_min_spl.sort_values('mean',ascending=False)

,col,mean,median
11,close/250ma_avg,0.047410,0.019947
1,close/120ma_avg,0.038664,0.011437
5,close/80ma_avg,0.036032,0.011981
7,close/250ma,0.030575,0.002375
4,close/70ma_avg,0.026232,0.004869
2,close/20ma_avg,0.023579,0.009046
9,close/30ma_avg,0.023253,0.004545
0,close/40ma_avg,0.023087,0.004995
17,close/50ma_avg,0.020208,0.004545
3,close/60ma_avg,0.019248,0.002287


,col,mean,median
11,close/250ma_avg,0.026932,0.015441
1,close/120ma_avg,0.024860,0.014706
5,close/80ma_avg,0.022287,0.011259
4,close/70ma_avg,0.017792,0.008197
9,close/30ma_avg,0.017549,0.016353
7,close/250ma,0.017152,0.004197
0,close/40ma_avg,0.016336,0.012475
2,close/20ma_avg,0.015459,0.018097
17,close/50ma_avg,0.013891,0.008443
3,close/60ma_avg,0.013191,0.007258


In [15]:
data_max_spc = [[col,df['mean'][:10].max(),df['mean'][:10].idxmax(),df['median'][:10].max(),df['median'][:10].idxmax()] for col,df in result_1000_spc.items()]
columns = ['col','max_mean','max_mean_bin','max_median','max_median_bin']
df_max_spc = pd.DataFrame(data_max_spc,columns=columns)
df_max_spc.sort_values('max_mean',ascending=False)

data_max_spl = [[col,df['mean'].max(),df['mean'].idxmax(),df['median'].max(),df['median'].idxmax()] for col,df in result_1000_spl.items()]
df_max_spl = pd.DataFrame(data_max_spl,columns=columns)
df_max_spl.sort_values('max_mean',ascending=False)

,col,max_mean,max_mean_bin,max_median,max_median_bin
11,close/250ma_avg,0.047410,"(0.3139, 0.4637]",0.019947,"(0.3139, 0.4637]"
15,close/10ma,0.039337,"(0.80099142, 0.82385952]",0.021142,"(0.80099142, 0.82385952]"
1,close/120ma_avg,0.038664,"(0.4371, 0.5734]",0.011437,"(0.4371, 0.5734]"
13,close/20ma,0.036349,"(0.8112, 0.8173]",0.012866,"(0.8112, 0.8173]"
5,close/80ma_avg,0.036032,"(0.4309, 0.6255]",0.011981,"(0.4309, 0.6255]"
6,close/5ma,0.033163,"(0.88279459, 0.88879642]",0.007484,"(0.88279459, 0.88879642]"
21,close/120ma,0.033021,"(0.671, 0.6775]",0.004637,"(0.671, 0.6775]"
8,close/10ma_avg,0.032204,"(0.82721, 0.83543]",0.012714,"(0.79719, 0.81596]"
10,close/30ma,0.030885,"(0.7677, 0.7783]",0.007527,"(0.7783, 0.7862]"
7,close/250ma,0.030575,"(0.2846, 0.4759]",0.002375,"(0.2846, 0.4759]"


,col,max_mean,max_mean_bin,max_median,max_median_bin
15,close/10ma,0.028609,"(0.80099142, 0.82385952]",0.022740,"(0.80099142, 0.82385952]"
11,close/250ma_avg,0.026932,"(0.3139, 0.4637]",0.015441,"(0.3139, 0.4637]"
13,close/20ma,0.025207,"(0.8112, 0.8173]",0.017083,"(0.7794, 0.7936]"
1,close/120ma_avg,0.024860,"(0.4371, 0.5734]",0.014706,"(0.4371, 0.5734]"
6,close/5ma,0.024293,"(0.88279459, 0.88879642]",0.012642,"(0.88279459, 0.88879642]"
8,close/10ma_avg,0.024054,"(0.82721, 0.83543]",0.020467,"(0.79719, 0.81596]"
2,close/20ma_avg,0.022330,"(0.77, 0.7826]",0.018097,"(0.4867, 0.7494]"
5,close/80ma_avg,0.022287,"(0.4309, 0.6255]",0.011259,"(0.4309, 0.6255]"
10,close/30ma,0.021325,"(0.7677, 0.7783]",0.013989,"(0.7677, 0.7783]"
21,close/120ma,0.020073,"(0.671, 0.6775]",0.007178,"(0.6545, 0.6635]"


In [17]:
metric = 'close/250ma_avg'
result_1000_spc[metric].head(20)
mask = (df[metric]<0.51)
index = df.index[mask]
len(index)
df_r_spc.loc[index].reset_index('code')['r'].resample('Q').agg(agg_operations)
df_r_spl.loc[index].reset_index('code')['r'].resample('Q').agg(agg_operations)
df_r_spl.loc[index].sort_values('date',ascending=False).head(20)

,mean,median,cnt
bin,,,
"(0.3139, 0.4637]",0.047410,0.019947,1772
"(0.4637, 0.4867]",0.031333,0.010101,1783
"(0.4867, 0.5016]",0.031388,0.008097,1782
"(0.5016, 0.5125]",0.031994,0.007680,1778
"(0.5125, 0.5212]",0.028020,0.001471,1782
"(0.5212, 0.5283]",0.025290,0.001163,1783
"(0.5283, 0.5346]",0.023568,-0.002443,1778
"(0.5346, 0.5403]",0.021069,-0.007858,1783
"(0.5403, 0.5456]",0.021161,-0.003933,1777


6678

,mean,median,max,min,std,size
date,,,,,,
2017-06-30,0.035038,0.007540,0.270130,-0.117320,0.090757,192
2017-09-30,0.042049,0.019960,0.471292,-0.125563,0.090382,395
2017-12-31,-0.007313,-0.013660,0.198738,-0.103969,0.054553,244
2018-03-31,0.076168,0.077074,0.535491,-0.153012,0.099631,960
2018-06-30,0.008252,-0.006969,0.264591,-0.163749,0.076004,419
2018-09-30,-0.006521,-0.019481,0.383729,-0.168421,0.066078,1506
2018-12-31,0.054496,0.033553,2.240000,-0.159780,0.114272,2276
2019-03-31,0.069641,0.046174,0.333333,-0.148311,0.114064,263
2019-06-30,0.003713,-0.010145,0.146826,-0.051360,0.053029,14


,mean,median,max,min,std,size
date,,,,,,
2017-06-30,0.017522,0.003794,0.267951,-0.117320,0.066080,192
2017-09-30,0.026150,0.019723,0.443662,-0.125563,0.069064,395
2017-12-31,-0.002924,-0.014717,0.134361,-0.103969,0.050506,244
2018-03-31,0.050386,0.049086,0.488167,-0.136153,0.075436,960
2018-06-30,0.013820,0.008108,0.264591,-0.163749,0.067828,419
2018-09-30,-0.002423,-0.011320,0.309824,-0.151862,0.056538,1506
2018-12-31,0.024754,0.015504,2.240000,-0.159780,0.086322,2276
2019-03-31,0.045753,0.017323,0.333333,-0.148311,0.095390,263
2019-06-30,0.021951,0.007902,0.146826,-0.051360,0.066307,14


buy_at        max  max_idx   idx  is_selled  sell_at  \
code      date                                                                  
600175.SH 2020-04-23        NaN        NaN      NaN  2316      False      NaN   
300362.SZ 2020-04-23        NaN        NaN      NaN  1142      False      NaN   
000669.SZ 2020-04-23        NaN        NaN      NaN  2272      False      NaN   
600734.SH 2020-04-23        NaN        NaN      NaN  1951      False      NaN   
002176.SZ 2020-04-23        NaN        NaN      NaN  2359      False      NaN   
300156.SZ 2020-04-23        NaN        NaN      NaN  2003      False      NaN   
603318.SH 2020-04-23        NaN        NaN      NaN  1051      False      NaN   
300431.SZ 2020-04-23        NaN        NaN      NaN  1007      False      NaN   
          2020-04-22   6.107640   6.477800   1007.0  1006      False      NaN   
300156.SZ 2020-04-22   9.927361   9.927361   2003.0  2002      False      NaN   
300362.SZ 2020-04-22   4.743160   4.773180   1142.0  1141      False      NaN   
600175.SH 2020-04-22  17.875200  18.045439   2316.0  2315      False      NaN   
603318.SH 2020-04-22  21.940859  21.940859   1051.0  1050      False      NaN   
000669.SZ 2020-04-22   8.178240   8.214750   2272.0  2271      False      NaN   
600734.SH 2020-04-22  12.440600  12.660140   1951.0  1950      False      NaN   
000669.SZ 2020-04-21   7.849650   8.579849   2271.0  2270       True      NaN   
603318.SH 2020-04-21  21.729599  22.001219   1050.0  1049      False      NaN   
600175.SH 2020-04-21  17.534719  18.215681   2315.0  2314      False      NaN   
300362.SZ 2020-04-21   4.833220   4.833220   1141.0  1140       True      NaN   
300156.SZ 2020-04-21  10.386960  10.386960   2002.0  2001       True      NaN   

                     sell_date   r  
code      date                      
600175.SH 2020-04-23       NaT NaN  
300362.SZ 2020-04-23       NaT NaN  
000669.SZ 2020-04-23       NaT NaN  
600734.SH 2020-04-23       NaT NaN  
002176.SZ 2020-04-23       NaT NaN  
300156.SZ 2020-04-23       NaT NaN  
603318.SH 2020-04-23       NaT NaN  
300431.SZ 2020-04-23       NaT NaN  
          2020-04-22       NaT NaN  
300156.SZ 2020-04-22       NaT NaN  
300362.SZ 2020-04-22       NaT NaN  
600175.SH 2020-04-22       NaT NaN  
603318.SH 2020-04-22       NaT NaN  
000669.SZ 2020-04-22       NaT NaN  
600734.SH 2020-04-22       NaT NaN  
000669.SZ 2020-04-21       NaT NaN  
603318.SH 2020-04-21       NaT NaN  
600175.SH 2020-04-21       NaT NaN  
300362.SZ 2020-04-21       NaT NaN  
300156.SZ 2020-04-21       NaT NaN

In [21]:
metric = 'close/10ma'
result_1000_spc[metric].head(20)
mask = (df[metric]>0.8) & (df[metric]<0.87) 
index = df.index[mask]
len(index)
df_r_spc.loc[index].reset_index('code')['r'].resample('Q').agg(agg_operations)
df_r_spl.loc[index].reset_index('code')['r'].resample('Q').agg(agg_operations)
df_r_spl.loc[index].sort_values('date',ascending=False).head(20)

,mean,median,cnt
bin,,,
"(0.58091284, 0.80099142]",-0.018508,-0.011921,3279
"(0.80099142, 0.82385952]",0.039337,0.021142,3279
"(0.82385952, 0.83601029]",0.035842,0.018594,3280
"(0.83601029, 0.84453193]",0.037776,0.017950,3284
"(0.84453193, 0.85111993]",0.037142,0.017782,3277
"(0.85111993, 0.85636391]",0.035403,0.014568,3282
"(0.85636391, 0.86094795]",0.033331,0.011805,3278
"(0.86094795, 0.86473148]",0.031898,0.011174,3284
"(0.86473148, 0.86818056]",0.031038,0.008568,3280


4894

,mean,median,max,min,std,size
date,,,,,,
2016-03-31,0.062190,0.043012,0.369879,-0.135593,0.086538,788
2016-06-30,0.007135,-0.012424,0.257862,-0.116116,0.094889,40
2016-09-30,-0.020808,-0.020808,-0.002331,-0.039286,0.026131,2
2016-12-31,-0.021882,-0.033721,0.105839,-0.109170,0.059088,19
2017-03-31,0.024656,-0.006183,0.271168,-0.271033,0.098370,77
2017-06-30,-0.012154,-0.020652,0.351604,-0.576541,0.093806,215
2017-09-30,0.012305,-0.013072,0.218978,-0.189972,0.073172,67
2017-12-31,-0.024612,-0.033238,0.297082,-0.332734,0.087587,101
2018-03-31,0.039126,0.009412,0.422652,-0.369876,0.104272,715


,mean,median,max,min,std,size
date,,,,,,
2016-03-31,0.066786,0.072961,0.369879,-0.135593,0.064533,788
2016-06-30,0.017293,-0.001432,0.238949,-0.091405,0.082974,40
2016-09-30,-0.018477,-0.018477,0.002331,-0.039286,0.029427,2
2016-12-31,0.005772,-0.005882,0.114963,-0.109170,0.064856,19
2017-03-31,0.010472,-0.006183,0.226772,-0.271033,0.077941,77
2017-06-30,-0.000378,-0.006110,0.248561,-0.145670,0.065553,215
2017-09-30,0.014025,-0.008753,0.203401,-0.189972,0.066734,67
2017-12-31,-0.004953,-0.007710,0.190175,-0.332734,0.076376,101
2018-03-31,0.000940,-0.007720,0.422652,-0.369876,0.070811,715


,,buy_at,max,max_idx,idx,is_selled,sell_at,sell_date,r
code,date,,,,,,,,
300540.SZ,2020-04-23,NaN,NaN,NaN,886,False,NaN,NaT,NaN
300457.SZ,2020-04-20,160.524139,161.738220,1143.0,1142,False,NaN,NaT,NaN
600250.SH,2020-04-17,15.013400,15.799300,2294.0,2291,False,NaN,NaT,NaN
300526.SZ,2020-04-15,252.374329,269.473206,766.0,764,True,253.763992,2020-04-20,0.005506
300820.SZ,2020-04-15,88.870003,93.000000,45.0,43,True,88.610001,2020-04-21,-0.002926
600250.SH,2020-04-14,16.205801,16.232899,2289.0,2288,True,15.230200,2020-04-17,-0.060201
300526.SZ,2020-04-14,244.137070,265.243774,765.0,763,True,261.820007,2020-04-17,0.072430
603557.SH,2020-04-13,8.160800,8.342600,635.0,633,True,8.029500,2020-04-22,-0.016089
603726.SH,2020-04-13,27.665638,28.164120,963.0,962,True,26.751759,2020-04-21,-0.033033


In [24]:
# metric = 'close/10ma'
metric2 = 'close/20ma'

# result_1000_spc[metric].head(20)
result_1000_spc[metric2].head(20)

# mask = (df[metric]>0.8) & (df[metric]<0.87) \
#     & (df[metric]>0.751) & (df[metric]<0.817)
mask = (df[metric]>0.751) & (df[metric]<0.817)
index = df.index[mask]
len(index)
df_r_spc.loc[index].reset_index('code')['r'].resample('Q').agg(agg_operations)
df_r_spl.loc[index].reset_index('code')['r'].resample('Q').agg(agg_operations)
df_r_spl.loc[index].sort_values('date',ascending=False).head(20)

,mean,median,cnt
bin,,,
"(0.3128, 0.751]",0.006935,-0.006865,3173
"(0.751, 0.7794]",0.032533,0.011917,3183
"(0.7794, 0.7936]",0.033655,0.011714,3188
"(0.7936, 0.8036]",0.031525,0.009972,3186
"(0.8036, 0.8112]",0.032677,0.011170,3184
"(0.8112, 0.8173]",0.036349,0.012866,3187
"(0.8173, 0.8224]",0.031742,0.008455,3188
"(0.8224, 0.8269]",0.028578,0.006464,3188
"(0.8269, 0.8309]",0.031563,0.007707,3189


4482

,mean,median,max,min,std,size
date,,,,,,
2016-03-31,0.065099,0.048901,0.347882,-0.135593,0.087632,647
2016-06-30,0.055130,-0.009690,0.238949,-0.066447,0.123284,12
2016-09-30,-0.039286,-0.039286,-0.039286,-0.039286,NaN,1
2016-12-31,-0.035669,-0.054270,0.036765,-0.100810,0.047684,10
2017-03-31,0.010470,-0.011900,0.256898,-0.153394,0.094874,48
2017-06-30,-0.021414,-0.024551,0.257069,-0.568826,0.084134,244
2017-09-30,0.013369,0.001845,0.218978,-0.189972,0.077916,58
2017-12-31,-0.026779,-0.041152,0.297082,-0.152047,0.074072,84
2018-03-31,0.026537,0.004118,0.454802,-0.604730,0.129298,678


,mean,median,max,min,std,size
date,,,,,,
2016-03-31,0.069643,0.076250,0.311258,-0.135593,0.069766,647
2016-06-30,0.052895,0.033816,0.238949,-0.052478,0.092725,12
2016-09-30,-0.039286,-0.039286,-0.039286,-0.039286,NaN,1
2016-12-31,0.003736,0.003873,0.088235,-0.066667,0.047739,10
2017-03-31,-0.011473,-0.013880,0.138872,-0.153394,0.057592,48
2017-06-30,-0.014642,-0.012777,0.164985,-0.259868,0.067472,244
2017-09-30,0.013791,-0.001047,0.203401,-0.189972,0.071304,58
2017-12-31,-0.012050,-0.014771,0.152078,-0.152047,0.060211,84
2018-03-31,-0.007010,-0.009259,0.422652,-0.604730,0.101229,678


buy_at         max  max_idx   idx  is_selled  \
code      date                                                           
600250.SH 2020-04-17   15.013400   15.799300   2294.0  2291      False   
          2020-04-16   15.230200   15.799300   2294.0  2290      False   
300526.SZ 2020-04-15  252.374329  269.473206    766.0   764       True   
300820.SZ 2020-04-15   88.870003   93.000000     45.0    43       True   
600250.SH 2020-04-15   15.528300   15.799300   2294.0  2289      False   
300820.SZ 2020-04-14   91.500000   92.339996     43.0    42       True   
300526.SZ 2020-04-14  244.137070  265.243774    765.0   763       True   
600250.SH 2020-04-13   15.528300   17.560801   2288.0  2287       True   
603557.SH 2020-04-13    8.160800    8.342600    635.0   633       True   
002953.SZ 2020-04-10   28.160101   29.567101    230.0   226       True   
300526.SZ 2020-04-10  251.750000  264.337494    763.0   761       True   
603726.SH 2020-04-10   27.437168   28.164120    963.0   961       True   
          2020-04-09   29.368780   29.555708    961.0   960       True   
300820.SZ 2020-04-07  105.000000  107.739998     38.0    37       True   
002953.SZ 2020-04-03   31.466549   32.391148    223.0   222       True   
600892.SH 2020-04-03   23.774479   24.849390   2326.0  2324       True   
603557.SH 2020-04-03    9.069799    9.201099    630.0   628       True   
002953.SZ 2020-04-02   30.260550   34.371002    222.0   221       True   
600753.SH 2020-04-02   29.984102   33.981979   2180.0  2169       True   
002506.SZ 2020-04-02   15.494880   15.851670   1870.0  1869       True   

                         sell_at  sell_date         r  
code      date                                         
600250.SH 2020-04-17         NaN        NaT       NaN  
          2020-04-16         NaN        NaT       NaN  
300526.SZ 2020-04-15  253.763992 2020-04-20  0.005506  
300820.SZ 2020-04-15   88.610001 2020-04-21 -0.002926  
600250.SH 2020-04-15         NaN        NaT       NaN  
300820.SZ 2020-04-14   92.500000 2020-04-17  0.010929  
300526.SZ 2020-04-14  261.820007 2020-04-17  0.072430  
600250.SH 2020-04-13   15.528300 2020-04-16  0.000000  
603557.SH 2020-04-13    8.029500 2020-04-22 -0.016089  
002953.SZ 2020-04-10   28.893749 2020-04-17  0.026053  
300526.SZ 2020-04-10  244.137070 2020-04-15 -0.030240  
603726.SH 2020-04-10   26.751759 2020-04-21 -0.024981  
          2020-04-09   27.665638 2020-04-14 -0.057992  
300820.SZ 2020-04-07   97.300003 2020-04-10 -0.073333  
002953.SZ 2020-04-03   30.451500 2020-04-09 -0.032258  
600892.SH 2020-04-03   22.383419 2020-04-13 -0.058511  
603557.SH 2020-04-03    8.494100 2020-04-13 -0.063474  
002953.SZ 2020-04-02   31.245449 2020-04-08  0.032547  
600753.SH 2020-04-02   34.745621 2020-04-21  0.158801  
002506.SZ 2020-04-02   14.322570 2020-04-13 -0.075658

In [ ]:
# 